In [1]:
import pandas as pd
import os

In [2]:
# Define a func to return head and tail of a df

def ends(df,x=1):
    dfends=df.head(x).append(df.tail(x))
    return dfends

In [3]:
# Read the cleaned data and sort by symbol and by data

data_df=pd.read_csv(os.path.join('C:/Users/jack.jin/Documents/Market', 'SP500_FIVE_YR_Cleaned_Sorted.csv'),sep=',')
ends(data_df)

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Row Count
0,2015-01-02,41.180000,41.310001,40.369999,40.560001,38.733665,1529200.0,A,1246
609293,2019-12-12,123.690002,125.000000,123.389999,123.599998,123.599998,1069232.0,ZTS,1246


In [4]:
# Get past five day's individual daily return

def past_n_day_indiv_pct(n,df):    
    
    past_n_days_indiv_df=pd.DataFrame()    
                 
    for a in range(n):        

        past_n_days_indiv_df['t'+str(-(a))+'_pct']=pd.DataFrame.pct_change(df['Adj Close']).shift(a)

    
    return past_n_days_indiv_df

In [5]:
# Get past five day's cumulative return

def past_n_day_cumu_pct(n,df):    
    
    past_n_days_df=pd.DataFrame()    
                 
    for a in range(n):

        past_n_days_df['t'+str(-(a+1))+'_cumu_pct']=pd.DataFrame.pct_change(df['Adj Close'],a+1).shift(1)


    return past_n_days_df

In [6]:
# Get next five day's individual daily return

def next_n_day_indiv_pct(n,df):    
    
    next_n_days_indiv_df=pd.DataFrame()    
                 
    for a in range(n):        

        next_n_days_indiv_df['t'+str((a))+'_pct']=pd.DataFrame.pct_change(df['Adj Close']).shift(-(a))

    next_n_days_indiv_df.drop('t0_pct',1,inplace=True)
    return next_n_days_indiv_df

In [7]:
# Get next five day's cumulative return

def next_n_day_cumu_pct(n,df):    
       
    next_n_days_df=pd.DataFrame()    
                    
    for a in range(n):

                next_n_days_df['t'+str(a+1)+'_cumu_pct']=pd.DataFrame.pct_change(df['Adj Close'],a+1).shift(-(a+1))


    return next_n_days_df

In [8]:
# Put all above functions together

def symbols_ret(df):
    
    # Get the unique symbol list
    symbol_list = pd.unique(df['Symbol']).tolist()
    
    # Create an empty dataframe to host to-be processed data
    combo_df=pd.DataFrame()
    
    for sym in symbol_list:
        
        symdf=df.loc[df['Symbol'] == sym,]
        
        sym_ret=symdf.join(past_n_day_indiv_pct(6,symdf)).join(past_n_day_cumu_pct(5,symdf)).join(next_n_day_indiv_pct(6,symdf)).join(next_n_day_cumu_pct(5,symdf))
               
        combo_df=combo_df.append(sym_ret)
    
    return combo_df 

In [9]:
stock_ret=symbols_ret(data_df)
stock_ret_na_dropped=stock_ret.dropna(how='any')

In [10]:
ends(stock_ret_na_dropped,5)

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Row Count,t0_pct,...,t1_pct,t2_pct,t3_pct,t4_pct,t5_pct,t1_cumu_pct,t2_cumu_pct,t3_cumu_pct,t4_cumu_pct,t5_cumu_pct
6,2015-01-12,40.610001,40.720001,39.950001,40.110001,38.303940,2770800.0,A,1246,-0.011825,...,-0.013962,-0.012389,-0.026881,0.006314,-0.008366,-0.013962,-0.026178,-0.052356,-0.046373,-0.054351
7,2015-01-13,40.470001,40.700001,39.330002,39.549999,37.769146,2013100.0,A,1246,-0.013962,...,-0.012389,-0.026881,0.006314,-0.008366,0.006064,-0.012389,-0.038938,-0.032870,-0.040961,-0.035145
8,2015-01-14,39.029999,39.099998,38.209999,39.060001,37.301208,5134000.0,A,1246,-0.012389,...,-0.026881,0.006314,-0.008366,0.006064,0.039046,-0.026881,-0.020737,-0.028930,-0.023042,0.015105
9,2015-01-15,39.330002,39.410000,37.990002,38.009998,36.298496,2628900.0,A,1246,-0.026881,...,0.006314,-0.008366,0.006064,0.039046,-0.021185,0.006314,-0.002105,0.003946,0.043147,0.021047
10,2015-01-16,37.830002,38.459999,37.759998,38.250000,36.527687,3004000.0,A,1246,0.006314,...,-0.008366,0.006064,0.039046,-0.021185,0.008761,-0.008366,-0.002353,0.036601,0.014641,0.023529
609284,2019-11-29,121.290001,121.820000,120.099998,120.519997,120.519997,1064400.0,ZTS,1246,-0.009126,...,-0.005061,0.005171,-0.006554,0.003090,0.013404,-0.005061,0.000083,-0.006472,-0.003402,0.009957
609285,2019-12-02,120.870003,121.500000,118.809998,119.910004,119.910004,2050400.0,ZTS,1246,-0.005061,...,0.005171,-0.006554,0.003090,0.013404,-0.004026,0.005171,-0.001418,0.001668,0.015095,0.011008
609286,2019-12-03,119.639999,120.790001,119.260002,120.529999,120.529999,2115200.0,ZTS,1246,0.005171,...,-0.006554,0.003090,0.013404,-0.004026,0.012786,-0.006554,-0.003485,0.009873,0.005808,0.018668
609287,2019-12-04,121.010002,121.919998,119.629997,119.739998,119.739998,2566500.0,ZTS,1246,-0.006554,...,0.003090,0.013404,-0.004026,0.012786,0.006353,0.003090,0.016536,0.012444,0.025388,0.031902
609288,2019-12-05,119.199997,120.260002,118.000000,120.110001,120.110001,2771100.0,ZTS,1246,0.003090,...,0.013404,-0.004026,0.012786,0.006353,0.000324,0.013404,0.009325,0.022230,0.028724,0.029057


In [11]:
# after dropping na, count # of rows again to make sure all tickers have the same number of trading days
symbol_count=stock_ret_na_dropped.groupby(['Symbol']).size().reset_index(name='Row Count 2')
stock_no_na_count=pd.merge(stock_ret_na_dropped,symbol_count, on='Symbol',how='left')
ends(stock_no_na_count)

# Now the 1247 count is gone
stock_no_na_count.groupby('Row Count 2')['Symbol'].nunique()

Row Count 2
1235    489
Name: Symbol, dtype: int64

In [ ]:
stock_ret_na_dropped.to_csv(os.path.join('C:/Users/jack.jin/Documents/Market', 'SP500_FIVE_YR_Cleaned_Sorted_Ret.csv'),sep=',', index=False)